# 线性回归：
$$ Y = X  \cdot W + b$$

    

## 在实现的时候，我们需要加入误差  
即:
$$ Y = X  \cdot W + b + \epsilon$$

对于一个给定的训练集:
$$[X,Y]$$
其中X和Y一一对应  
我们需要做的是以下几步:  
一.预处理数据  
包括:清理空值、标签对应等等  
最重要的: 定义 **batch_size** ,按照 **batch_size** 进行随机取样  
这样我们就得到了若干X-Y样本
$$[X_1,Y_1][X_2,Y_2]\dots[X_i,Y_i] \quad where\;i = 1,2,\dots n$$


之后我们根据 原始回归函数的输出得到 $\hat{Y}$  
  
# Loss  
对每个batch来说 均方误:  
$$ loss\quad=\; \sum_{i=1}^{i=n} \hat{y_i} - y_i \quad where \; \hat{y_i},y_i\quad in \quad \hat{Y},Y $$

# SGD  
优化的本质:调整公式中的参数，让下一次训练产生的**损失更低**
方法:对损失求导，得到梯度grad，让参数-lr*梯度，得到新的参数值,以w为例:
$$w^n = w^n-1 - lr \cdot grad $$

考虑到我们的grad是一整个batch积累的总的值，我们要对他求均值，所以是：
$$w^n = w^n-1 - lr \cdot \bar{grad} $$
其中 $$ \bar{grad} = \frac{\sum_{i = 1}^{i=batch\_size} grad_i}{batch\_size}  $$



# epoch  
为了不断优化，我们应该重复以上操作 若干个epoch  
```pseudocode
for each_epoch
    for each_batch
        do cur_y = func(X)
        loss = Loss_comput(cur_y,y)
        w,b = SGD(lr,loss)

In [8]:
import torch
import random
#实战
#我们希望得到一个 y = Wx + b + sigma
def create_data(w,b,num_examples):
    #w的size是对应了特征维度的偏差
    #b也是
    #num_examples是样本数量
    X = torch.normal(0,1,(num_examples,len(w)))#先生成X
    y = torch.matmul(X,w)+b#根据x生成y//这里用的是matmul，所以X*w变成了一列，但是是一维的，我们希望它是二维的
    y += torch.normal(0,0.01,y.shape)#在加上偏差
    return X,y.reshape((-1,1))
    #y.reshape((-1, 1)) 确保标签 y 是列向量（形状为 (样本数, 1)），方便后续与模型输出（通常也是列向量）计算损失。
    #而 -1 是一个特殊值，它的含义是：“根据另一个维度的大小和总元素数，自动计算当前维度的大小”。

#初始化真实的数据
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = create_data(true_w,true_b,1000)



In [18]:
#随机初始化一个b和w
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


In [ ]:
#数据分批处理函数
def data_iter(batch_size, features, labels):
    num_examples = len(features)  # 获取总样本数（如1000个样本）
    indices = list(range(num_examples))  # 生成样本索引列表（如[0,1,2,...,999]）
    # 随机打乱索引：让样本顺序随机化，避免模型学习到数据顺序的规律
    random.shuffle(indices)
    # 按批次遍历所有样本：从0开始，每次跳batch_size步
    for i in range(0, num_examples, batch_size):
        # 取当前批次的索引：从i到i+batch_size（最后一批可能不足batch_size，用min处理）
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)]
        )
        # 用生成器（yield）返回当前批次的特征和标签
        yield features[batch_indices], labels[batch_indices]

In [12]:
#定义函数
def linreg(X,w,b):
    return torch.matmul(X,w) + b
#定义损失
def squared_loss(y_hat,y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2
#定义sgd
def sgd(params,lr,batch_size):
    """小批量随机梯度下降"""
    """所谓随机指的是每次更新的数据样本是随机选择的，而不是固定选择的"""
    with torch.no_grad():#临时关闭张量的梯度计算功能
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

# 开始训练

In [19]:
#定义一些基本的
lr = 0.3
num_epoch = 3
net = linreg
loss = squared_loss
#已知总数据大小是1000，我们取每批10个
batch_size = 10
for epoch in range(num_epoch):
    for X,y in data_iter(batch_size=batch_size,features = features,labels = labels):
        l = loss(net(X,w,b),y)
        l.sum().backward()
        sgd([w,b],lr,batch_size=batch_size)
    with torch.no_grad():
        train_loss = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1},loss {float (train_loss.mean()):f}')

epoch 1,loss 0.000050
epoch 2,loss 0.000049
epoch 3,loss 0.000049
